In [3]:
# Load libraries

# compare algorithms
import pandas as pd
import numpy as np
from pandas import read_csv
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))
df=read_csv('train.csv')
dftest=read_csv('test.csv')
df = df.replace(np.nan, 0)
df=df.replace('male','0')
df=df.replace('female','1')

df['Embarked']=df['Embarked'].replace('S','1')
df['Embarked']=df['Embarked'].replace('C','2')
df['Embarked']=df['Embarked'].replace('Q','3')
df=df.astype({'Sex':int,'Embarked':int})
df=df.drop(columns=['Name','Cabin','Ticket'])

dftest = dftest.replace(np.nan, 0)
dftest=dftest.replace('male','0')
dftest=dftest.replace('female','1')
dftest['Embarked']=dftest['Embarked'].replace('S','1')
dftest['Embarked']=dftest['Embarked'].replace('C','2')
dftest['Embarked']=dftest['Embarked'].replace('Q','3')
dftest=dftest.astype({'Sex':int,'Embarked':int})
dftest=dftest.drop(columns=['Name','Cabin','Ticket'])

x=df.drop(columns=['Survived'])
y=df['Survived']
# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
    cv_results = cross_val_score(model, x, y, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))
# Compare Algorithms
pyplot.boxplot(results, labels=names)
pyplot.title('Algorithm Comparison')
pyplot.show()    


# Make predictions on validation dataset
model = LogisticRegression()
model.fit(x, y)
predictions = model.predict(dftest)
pr=pd.DataFrame(dftest['PassengerId'],columns=['PassengerId'])
pr['survived']=predictions
pr.set_index('PassengerId', inplace=True)

pr.to_csv('prediction_results.csv')


PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object